# Dependences

In [1]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")
include("../../src/forces/forces.jl")

sum_force!

# Model

In [2]:
@time model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 150000.0,
        dt    = 0.5,
        nₖₙₙ  = 100,
        nₛₐᵥₑ = 50
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ""
    ) 
)

dump(model)

  0.000000 seconds (4 allocations: 144 bytes)


ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 150000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 100
    nₛₐᵥₑ: Int64 50
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String 

"../../data/init/Sphere"
  Output: OutputModel
    name_output: String "Test_1"
    path_output: String ""


# Aggregate

In [3]:
agg = nothing
# @time @start_agg FusionAGG = FusionAggregate(
@time agg = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(0.01,2.0,3.0), 
                ContractilePar(0.001)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    model
)
show_aggregates(agg)

  4.254377 seconds (13.10 M allocations: 1.097 GiB, 4.34% gc time, 4.38% compilation time)


2-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("HEK_2", InteractionPar(Cubic{Float64}(0.01, 2.0, 3.0), ContractilePar(0.001)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

========================= Type =======================


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

======================   Index =======================
Index of List of Aggregates


...

ContractilePar(Float32[0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001  …  0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001, 0.001])

Contractile


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 15.27

Radius


------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (1000, 5008)
idx_sum  = (1, 5008)
idx_cont = (100, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


# kNN

In [4]:
dist = zeros(size(agg.Simulation.Neighbor.idx_red)) |> cu

println("------------------------ SIZE ------------------------")
threads=(100)
@cuda(
    threads=threads,
    blocks=cld.(size(agg.Position,1),threads),
    dist_kernel!(
        agg.Simulation.Neighbor.idx_red,
        agg.Simulation.Neighbor.idx_cont,
        agg.Simulation.Neighbor.idx_sum,
        dist,
        agg.Position,
        agg.Simulation.Parameter.Force.rₘₐₓ
    )
)   

println("------------------------ IDX -------------------------")
display(agg.Simulation.Neighbor.idx_red)

------------------------ SIZE ------------------------


1000×5008 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 36  37  38  41  42  43  44  45  57  …  4965  4966  4967  4972  4973  4974
  6   7   8  49   1   2   3   3  58     5006  5007  5008  5002  5003  5004
  5   6   7  50  50   1   2  53  15     5007  5008  4959  5003  5004  5005
 42  43  44   5  51  51  52  54  10     4958  4959  4960  4965  4966  4967
 43  44  45  10   6  52  53   7  49     4957  4958  5004  4964  4965  4966
  2   3   2   9   4   7   8  14   4  …  5002  5003  4999  5007  5006  5007
 35   1  37  40  11   5   6  13  66     5004  5005  5000  4973  5008  4975
 37  36  39  42  10  12  13  44  50     4997  4998  4966  4971  4972  4973
 51  38  53  58  41  11  12  46  48     4998  4999  4968  4957  4974  4959
  0  52   0   0  43  42  43  62   0     4964  4965  4951     0  4958     0
  ⋮                   ⋮              ⋱                       ⋮        
  0   0   0   0   0   0   0   0   0        0     0     0     0     0     0
  0   0   0   0   0   0   0   0   0        0     0  

------------------------ IDX -------------------------


# Force

## Function

In [311]:
points    = copy(agg.Position)

idx_sum   = agg.Simulation.Neighbor.idx_sum
idx       = agg.Simulation.Neighbor.idx_red
pol       = agg.Simulation.Force.Pol

force     = agg.Simulation.Force.F
force_par = agg.Simulation.Parameter.Force
cont_par  = agg.Simulation.Parameter.Contractile.fₚ
dt        = model.Time.dt

function teso!(points,force,pol,idx_sum,idx, force_par, cont_par, dt)

    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    ti, tk = threadIdx().x, threadIdx().y

    norm = @cuDynamicSharedMem(Float32,(ti,tk))
    N_i  = @cuDynamicSharedMem(Float32,(ti))
    N_j  = @cuDynamicSharedMem(Float32,(ti))

    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning force
        force[i,k] = 0

        # # Generating Polarization vector
        phi = 2*pi*(2*rand() - 1)
        pol[i,3] = 2*rand() - 1
        pol[i,1] = sqrt(1-pol[i,3]^2)*cos(phi)
        pol[i,2] = sqrt(1-pol[i,3]^2)*sin(phi)

        # Iterate on each row
        for j=1:idx_sum[i]
            if idx[j,i] != 0
                # Finding forces
                dist = euclidean(points,idx[j,i],i)
                norm[ti,tk] = (points[i,k]-points[idx[j,i],k])/dist
                sync_threads()
                if dist < force_par.rₘₐₓ[i]
                    force[i,k] += force_func(force_par,i,dist) * norm[ti,tk]
                    sync_threads()
                end

                # Adding Contractile Forces
                N_i[ti] =  (points[i,1]-points[idx[j,i],1])/dist - pol[i,1]
                          +(points[i,2]-points[idx[j,i],2])/dist - pol[i,2]
                          +(points[i,3]-points[idx[j,i],3])/dist - pol[i,3]
                N_j[ti] =  (points[i,1]-points[idx[j,i],1])/dist - pol[idx[j,i],1]
                          +(points[i,2]-points[idx[j,i],2])/dist - pol[idx[j,i],2]
                          +(points[i,3]-points[idx[j,i],3])/dist - pol[idx[j,i],3]

                sync_threads()


            end
        end

        # points[i,k] += force[i,k] * dt

    end

    return nothing

end

teso! (generic function with 1 method)

In [330]:
threads=(32,3)
blocks = cld.(size(points),threads)
@cuda(
    threads = threads,
    blocks = blocks,
    shmem=prod(threads.+2)*sizeof(Float32),
    teso!(points,force,pol,idx_sum,idx,force_par, cont_par, dt)
)
display(force)

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
  0.80716     0.80716     0.80716
 -0.671855   -0.671855   -0.671855
  0.483491    0.483491    0.483491
  0.475346    0.475346    0.475346
  0.0431795   0.0431795   0.0431795
 -0.771118   -0.771118   -0.771118
 -0.466263   -0.466263   -0.466263
 -0.610903   -0.610903   -0.610903
  0.873207    0.873207    0.873207
  1.02757     1.02757     1.02757
  ⋮                      
 -0.212333   -0.212333   -0.212333
 -0.0777857  -0.0777857  -0.0777857
 -0.562875   -0.562875   -0.562875
 -0.125327   -0.125327   -0.125327
 -0.55412    -0.55412    -0.55412
 -0.421884   -0.421884   -0.421884
 -0.761701   -0.761701   -0.761701
  0.31742     0.31742     0.31742
  0.539082    0.539082    0.539082

## CPU

In [8]:
# X_CPU          = Matrix(FusionAGG.Position)
# idx_red_CPU    = Matrix(FusionAGG.Simulation.Neighbor.idx_red)
# idx_cont_CPU   = Matrix(FusionAGG.Simulation.Neighbor.idx_cont)
# force_CPU      = zeros(size(X_CPU))

# Param = vcat(
#         [
#             unique(
#                 getfield(FusionAGG.Simulation.Parameter.Force, fieldnames(Cubic)[i])
#             ) for i=1:size(fieldnames(Cubic),1)
#     ]...)
# Param          = Cubic(Param...)
# A_con          = unique(FusionAGG.Simulation.Parameter.Contractile.fₚ)[1]

# t_knn = 1

In [9]:
# function runCPU()
#     for j = 1:size(idx_red_CPU,1)
#         for i =1:size(X_CPU,1)
#             if idx_red_CPU[j,i] != 0 && idx_red_CPU[j,i] != i
#                 dist = euclidean(X_CPU,i,idx_red_CPU[j,i])
#                 force_CPU[i,:] += force_func(Param,dist) .* (X_CPU[i,:] - X_CPU[idx_red_CPU[j,i],:]) ./ dist
#             end
#         end
#     end

#     for i =1:size(X_CPU,1)
#         if idx_cont_CPU[t_knn,i] != i
#             dist = euclidean(X_CPU,i,idx_cont_CPU[t_knn,i])
#             force_CPU[i,:] += A_con .* (X_CPU[i,:] - X_CPU[idx_cont_CPU[t_knn,i],:]) ./ dist
#         end
#     end
# end

# runCPU()
# display(force_CPU)

## GPU vs CPU

In [10]:
# using Test
# @test Matrix(FusionAGG.Simulation.Force.F) ≈ force_CPU atol=0.01